<a href="https://colab.research.google.com/github/dustin-kang/Proj4_LyricsGenerator/blob/main/Note/%5BP403%5Ddata_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛠 데이터 전처리
- 크롤링한 데이터를 분석하는 작업이다.
- 추가적인 시각화도 한다.

## 0. P402 사전 작업

In [19]:
import pandas as pd
import numpy as np
import nltk

# 시각화 도구 #
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image # 편지지 시각화
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator 
from PIL import Image, ImageDraw, ImageFont

# warnings #
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/AI07/proj/vibe_j.csv')
nltk.download('punkt')
df['chars'] = df['lyrics'].apply(len)
df['words'] = df.apply(lambda row: nltk.word_tokenize(row['lyrics']), axis=1).apply(len)
df['lines'] = df['lyrics'].str.split('\n').apply(len)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 1. 말뭉치(Corpus) 생성 및 데이터 전처리

In [32]:
corpus = ''

for doc in df.lyrics:
  corpus += doc # 곡 당 가사들을 corpus 문자열에 넣는다.

corpus = corpus.lower()

print("유니크한 철자의 수:", len(set(corpus)))
print("유니크한 철자:",sorted(set(corpus)))

유니크한 철자의 수: 511
유니크한 철자: ['\n', ' ', '!', '"', '$', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '~', '¡', '´', '¾', '¿', 'à', 'á', 'å', 'æ', 'é', 'ê', 'í', 'î', 'ñ', 'ó', 'ø', 'ú', 'е', '\u2005', '\u200a', '–', '—', '‘', '’', '‚', '“', '”', '…', '′', '″', '※', '\u205f', '가', '각', '간', '갈', '감', '갑', '갖', '같', '개', '객', '거', '걱', '건', '걸', '것', '게', '겐', '겠', '겨', '결', '겹', '곁', '계', '고', '곡', '곧', '곳', '공', '과', '관', '괜', '괴', '교', '구', '국', '군', '굴', '궁', '귀', '그', '극', '근', '금', '기', '긴', '길', '깊', '까', '깐', '깨', '꺼', '껏', '께', '꽃', '꾸', '꿈', '끊', '끝', '끼', '낄', '나', '난', '날', '남', '났', '낯', '내', '냐', '너', '넌', '널', '네', '녀', '녕', '노', '놀', '놈', '놓', '누', '눈', '눌', '느', '는', '늘', '니', '닌', '닐', '다', '닥', '단', '닫', '달', '당', '대', '더', '덕', '던', '도', '돌', '동', '돼

In [33]:
import re
corpus = re.sub("[^A-Za-z0-9'\.\n]"," ",corpus)

In [34]:
print("유니크한 철자:",sorted(set(corpus)))

유니크한 철자: ['\n', ' ', "'", '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


## 2. 고유 문자 목록 만들기

In [45]:
# 딕셔너리를 매핑하기 위해 말뭉치에 있는 모든 Unique_char(고유문자)을 저장한다.
unique_char = sorted(list(set(corpus)))

corpus_len = len(corpus)
unique_len = len(unique_char)

# 어휘에 접근할 딕셔너리를 만든다. -> unique_char과 매핑
mapping = dict((c, i) for i, c in enumerate(unique_char))
re_mapping = dict((i, c) for i, c in enumerate(unique_char)) # 역(반대)

print("문자열 철자의 총 갯수:", corpus_len)
print("고유문자(unique_len)의 갯수:", unique_len)

문자열 철자의 총 갯수: 3846836
고유문자(unique_len)의 갯수: 40


In [52]:
# 코퍼스 분할
length = 40
features = []
targets = []

for i in range(0, corpus_len - length):
    feature = corpus[i:i + length]
    target = corpus[i + length]
    features.append([mapping[j] for j in feature])
    targets.append(mapping[target])

target_len = len(targets)
print("말뭉치 내 시퀀수의 총합:", target_len)

말뭉치 내 시퀀수의 총합: 3846796


## 3. Features 와 Target 데이터 만들기

In [54]:
from keras.utils import np_utils

# Features로 Reshape 후 고유문자로 정규화 진행
X = (np.reshape(features, (target_len, length, 1)))/ float(unique_len)

# Target값 원핫 인코딩
## keras.utils.np_utils 패키지에 있는 to_categorical 함수는 바로 One-hot 인코딩을 해주는 함수입니다.
y = np_utils.to_categorical(targets)